In [1]:
import numpy as np
import pandas as pd
import scipy

In [2]:
core_atom_array=np.zeros((1,3))

In [3]:
from numpy import cross, eye, dot
from scipy.linalg import expm, norm

def M(axis, theta):
    return expm(cross(eye(3), axis/norm(axis)*theta))

v, axis, theta = np.array([0,0.56*np.cos(27.6*np.pi/180),0.56*np.sin(27.6*np.pi/180)]), [0,0,1], np.pi*2/3
M0 = M(axis, theta)


def rotate(v,axis,theta):
    return dot(M(axis,theta),v)

In [4]:
dipole_atom_array=np.zeros((6,3))

In [5]:
dipole_atom_array[0]=np.array([0,1*np.cos(27.6*np.pi/180),1*np.sin(27.6*np.pi/180)])
dipole_atom_array[1]=np.array([0,1*np.cos(27.6*np.pi/180),-1*np.sin(27.6*np.pi/180)])
dipole_atom_array[2]=rotate(dipole_atom_array[0],[0,0,1],np.pi*2/3)
dipole_atom_array[3]=rotate(dipole_atom_array[1],[0,0,1],np.pi*2/3)
dipole_atom_array[4]=rotate(dipole_atom_array[0],[0,0,1],np.pi*4/3)
dipole_atom_array[5]=rotate(dipole_atom_array[1],[0,0,1],np.pi*4/3)


In [6]:
2.2*np.sin(27.6*np.pi/180)

1.019251277263696

In [7]:
fl_atom_array=np.zeros((24,3))

In [8]:
#build fl atoms
fl_atom_array[0]=dipole_atom_array[0]*0.8
fl_atom_array[1]=dipole_atom_array[0]+rotate(dipole_atom_array[0]*(-0.2),[1,0,0],np.pi/2)
fl_atom_array[2]=rotate(fl_atom_array[1],dipole_atom_array[0],np.pi*2/3)
fl_atom_array[3]=rotate(fl_atom_array[1],dipole_atom_array[0],np.pi*4/3)
for i in range(4,8):
    fl_atom_array[i]=rotate(fl_atom_array[i-4],[0,1,0],np.pi)
for i in range(1,3):
    for j in range(0,8):
        fl_atom_array[8*i+j]=rotate(fl_atom_array[j],[0,0,1],np.pi*2*i/3)

In [9]:
fl_atom_array

array([[ 0.00000000e+00,  7.08962863e-01,  3.70636828e-01],
       [ 0.00000000e+00,  9.78862786e-01,  2.86055319e-01],
       [-1.73205081e-01,  8.39873976e-01,  5.51916393e-01],
       [ 1.73205081e-01,  8.39873976e-01,  5.51916393e-01],
       [ 5.65944292e-17,  7.08962863e-01, -3.70636828e-01],
       [ 4.36792469e-17,  9.78862786e-01, -2.86055319e-01],
       [ 1.73205081e-01,  8.39873976e-01, -5.51916393e-01],
       [-1.73205081e-01,  8.39873976e-01, -5.51916393e-01],
       [-6.13979850e-01, -3.54481432e-01,  3.70636828e-01],
       [-8.47720040e-01, -4.89431393e-01,  2.86055319e-01],
       [-6.40749659e-01, -5.69936988e-01,  5.51916393e-01],
       [-8.13954739e-01, -2.69936988e-01,  5.51916393e-01],
       [-6.13979850e-01, -3.54481432e-01, -3.70636828e-01],
       [-8.47720040e-01, -4.89431393e-01, -2.86055319e-01],
       [-8.13954739e-01, -2.69936988e-01, -5.51916393e-01],
       [-6.40749659e-01, -5.69936988e-01, -5.51916393e-01],
       [ 6.13979850e-01, -3.54481432e-01

In [10]:
data = {'Atomid': [],
        'Mol': [],
        'Type': [],
       'q':[],'x':[],'y':[],'z':[],'vx':[],'vy':[],'vz':[]}
atoms = pd.DataFrame(data)

In [11]:
#add info to atoms dataframe
for i in range(0,core_atom_array.shape[0]):
    atoms.loc[len(atoms.index)] = [i+1, 1, 1,0,core_atom_array[i,0],core_atom_array[i,1],core_atom_array[i,2],0,0,0]
for i in range(0,dipole_atom_array.shape[0]):
    atoms.loc[len(atoms.index)] = [i+core_atom_array.shape[0]+1, 1, 2,0,dipole_atom_array[i,0],dipole_atom_array[i,1],dipole_atom_array[i,2],0,0,0]
for i in range(0,fl_atom_array.shape[0]):
    atoms.loc[len(atoms.index)] = [i+core_atom_array.shape[0]+dipole_atom_array.shape[0]+1, 1, 4,0,fl_atom_array[i,0],fl_atom_array[i,1],fl_atom_array[i,2],0,0,0]

In [12]:
def replicate_molecules(atoms,dx,dy,dz,thetax,thetay,thetaz):
    #position renew
    atoms_copy=atoms.copy()
    
    shiftx=atoms_copy.loc[0,'x']
    atoms_copy['x']-=shiftx
    
    shifty=atoms_copy.loc[0,'y']
    atoms_copy['y']-=shiftx
   
    shiftz=atoms_copy.loc[0,'z']
    atoms_copy['z']-=shiftz
    posarray=atoms_copy.iloc[:,4:7].to_numpy()
    rx=M([1,0,0],thetax)
    b=[rx]*len(atoms_copy.index)
    rx_all=scipy.linalg.block_diag(*b)
    ry=M([0,1,0],thetay)
    b=[ry]*len(atoms_copy.index)
    ry_all=scipy.linalg.block_diag(*b)
    rz=M([0,0,1],thetaz)
    b=[rz]*len(atoms_copy.index)
    rz_all=scipy.linalg.block_diag(*b)
    newposarray=np.reshape(np.matmul(rz_all,np.matmul(ry_all,np.matmul(rx_all,posarray.flatten()))),[-1,3])
    atoms_copy.iloc[:,4:7]=newposarray
    atoms_copy['x']+=shiftx+dx
    atoms_copy['y']+=shifty+dy
    atoms_copy['z']+=shiftz+dz
    return atoms_copy

In [13]:
#build nanotube
#build one unit cell
atoms=replicate_molecules(atoms,0,0,0,0,0,-np.pi/2)
mol_1=atoms
mol_2=replicate_molecules(atoms,0,1.95*np.sqrt(3),0,0,0,0)
mol_3=replicate_molecules(atoms,1.95*3/2,1.95*np.sqrt(3)/2,0,0,0,0)
mol_4=replicate_molecules(atoms,-1.95/2,1.95*np.sqrt(3)/2,1.02,0,0,np.pi/3)
mol_5=replicate_molecules(atoms,1.95,0,1.02,0,0,np.pi/3)
mol_6=replicate_molecules(atoms,1.95,1.95*np.sqrt(3),1.02,0,0,np.pi/3)
atoms=pd.concat([mol_1,mol_2,mol_3,mol_4,mol_5,mol_6],ignore_index=True)
cell1=atoms
for i in range(1,10):
    cell=replicate_molecules(cell1,0,0,2.04*i,0,0,0)
    atoms=pd.concat([atoms,cell],ignore_index=True)

In [14]:
for i in range(0,60):
    atoms.loc[31*i:31*i+31,'Mol']=i+1

In [36]:
import random
for i in range(0,60):
    for j in range(0,6):
        a=random.randint(0,1)
        if a==1:
            atoms.loc[31*i+j+1,'Type']=3

In [43]:
atoms[0:32]

,Atomid,Mol,Type,q,x,y,z,vx,vy,vz
0,1.0,1.0,1.0,0.0,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0
1,2.0,1.0,3.0,0.0,0.886204,0.000000e+00,0.463296,0.0,0.0,0.0
2,3.0,1.0,3.0,0.0,0.886204,0.000000e+00,-0.463296,0.0,0.0,0.0
3,4.0,1.0,3.0,0.0,-0.443102,7.674748e-01,0.463296,0.0,0.0,0.0
4,5.0,1.0,3.0,0.0,-0.443102,7.674748e-01,-0.463296,0.0,0.0,0.0
5,6.0,1.0,3.0,0.0,-0.443102,-7.674748e-01,0.463296,0.0,0.0,0.0
6,7.0,1.0,2.0,0.0,-0.443102,-7.674748e-01,-0.463296,0.0,0.0,0.0
7,8.0,1.0,4.0,0.0,0.708963,0.000000e+00,0.370637,0.0,0.0,0.0
8,9.0,1.0,4.0,0.0,0.978863,0.000000e+00,0.286055,0.0,0.0,0.0
9,10.0,1.0,4.0,0.0,0.839874,1.732051e-01,0.551916,0.0,0.0,0.0


In [141]:
import plotly.express as px

fig = px.scatter_3d(atoms.loc[0:31*12], x='x', y='y', z='z',
              color='Type')


fig.show()

In [145]:
atoms.loc[0:10,'x']

0     0.000000
1     0.886204
2     0.886204
3    -0.443102
4    -0.443102
5    -0.443102
6    -0.443102
7     0.708963
8     0.978863
9     0.839874
10    0.839874
Name: x, dtype: float64

In [47]:
#write head of data file
f=open("data.pet1fltwotypenanotube",'w')
f.write("Lammps description\n")
f.write("\n")
f.write("1860 atoms\n")
f.write("4 atom types\n")
f.write("0 10 xlo xhi\n")
f.write("0 10 ylo yhi\n")
f.write("0 30 zlo zhi\n")


13

In [48]:
#write atoms
f.write("\n")
f.write("Atoms # full\n")
f.write("\n")
for i in range(0,1860):
    f.write("%d %d %d %f %f %f %f\n"%(i+1,atoms.loc[i,'Mol'],atoms.loc[i,'Type'],atoms.loc[i,'q'],atoms.loc[i,'x'],atoms.loc[i,'y'],atoms.loc[i,'z']))

In [49]:
f.close()

In [24]:
2**(1/6)

1.122462048309373

In [25]:
0.5/1.12

0.4464285714285714